# Part 1 - Hive Modeling
# Part 2 - Optimization

Sept 23 - lab 8  
Sept 25 - lab 9

<a href="#Creating-databases">Create Database</a>  
<a href="#Code-Syntax-(Create-and-Load)">Create Database (Syntax)</a>  
<a href="#Code-Syntax-(Working-with-database)">Work with Database (Syntax)</a>  
<a href="#Optimizing-Hive-Query-Performance">Query Optimize</a>  

---

# Creating databases

Managed database will be saved to `user/hive/warehouse/dualcore.db` and each table will have sub dir inside this directory  
Tables = stored in multiple files `user/hive/warehouse/dualcore.db/table`

External tables need to specify where we are storing the file


**Data validation**  
"Schema on read" meaning it wont check for errors when file format is incorrect  
Null can be either true null or just our mistake


**Table format (default)**  
Plain text  
One record per line by `\n`  
Column split by `A^`  
$\quad$ `B^` if members of arrays or structs  
$\quad$ `C^` if key value pairs

**Row format (default)**  
Delimited  
Serde (text file formats)

---

# Optimizing Hive Query Performance

### Use a faster execution engine (default is mapreduce)

Tez - more flexible `set hive.execution.engine=tez;`  
Spark - uses more memory but is faster than mapreduce `set hive.execution.engine=spark;`



### Use faster storage formats (default is textfile)

Depends on...  
Ingest pattern - where we get our data from  
Tool compatibility  
Expected lifetime  
Storage and performance reqs

<img src=https://i.imgur.com/KQ31PM9.png width="400" height="340" align="left">


**Binary encoding**  
Stores data as key-value pairs

**Columnar**   
Data stored down the column, not across the rows ("stored by column")  
Pro: efficient when we only need a subset of table (doesn't have to store duplicates)

**Textfile (Default)**  
Pro: Read or written from pretty much and programming language  
Con: Not great at scale - numbers are stored as strings, hard to represent binary

**1. SequenceFiles (binary encoding)** store key-value pairs  
Pro: Much more efficient  
Con: Doesn't work well with other tools, Java specific

**2. Apache Avro (binary encoding)**  
Pro: efficient, supported thruout Hadoop, good for long term storage  

**3. Apache Parquet (columnar and binary encoding)**  
Pro: reduced storage performance and improved performance, best when adding many records at once

**4. RCFile (columnar and binary encoding)** stores different row groups into a columnar table  
Con: pretty poor performance

**5. ORCFile (columnar and binary encoding)** stores different row groups into a columnar table  
Pro: better performance than RCFile, works well with Hive and Spark

`CREATE TABLE tab2 (
    order_id INT,
    prod_id INT)
    STORED AS PARQUET` or whatever format we want
    
`INSERT OVERWRITE TABLE tab1
    SELECT * FROM TAB2;`

---

# Optimizing Hive Query Performance (pt2)


### Partitions - only read part of the data

Subdivides data based on values  
Partioned field is removed from table

**Use when**  
Reading takes too long  
Queries typically filter on partitioned columns  
Not too many different values per part col  
When data comes in by month or something like that  
Part cols need to be in the data


**Example**  
Web log data, 100gb per day. Want to run ML on the data.  
Should partition table on the day  
Would be better with static with a macro to run the partition every day  
(Note that dynamic is slower because Hive needs to look into the file)  
(If we were partioning by state, dynamic would be better)


**Example**  
Hive dataset (5tb). Want to take samples and run ML on.  
Approach to obtain sample - bucketing (gives randomness based on hash)  
Optimization technique - use sort by with a random generator for id


---

### Loading data into partitioned tables

To enable dynamic  
hive.exec.dynamic.partition true  
hive.exec.dynamic.partition.mode nonstrict

#### 1 - Dynamic - automatic done based on col values  

**When creating table**  
`create external table __ (
partitioned by (state STRING)
...  `


**When altering table**  
`insert overwrite table __
partition column
select col, col, partition_col 
from __`


#### 2 - Static - manually creating partitions  

**Adding**  
`alter table __
add partition (col=val)`

**Dropping**  
`alter table __
drop partition (col=val)`



---

### View partitions

`show partitions table;`

---

### Better code

Filter rows early, only show relevant cols  
Use `explain` before query to show query plan 

`order by` uses single reducer (global sort)  
`sort by` sorts input before going to reducer **preferred**  

---

### Bucketing - subdivide data

Best when we want samples  
Data into separate **files**  
Partition was into separate fields

# Lab 8 (Sept 23) - How do we create Hive Tables?



https://pages.github.umn.edu/deliu/bigdata19/04-Hive2/lab08-datamodel.html  
https://pages.github.umn.edu/deliu/bigdata19/04-Hive2/lab08-datamodel-solution.html


---

### Step 1 - Create table

First way - using Sqoop because data already in mySQL  
We are creating the data + metadata here  
Because we created, this is managed table

`sqoop import \`  
` --connect jdbc:mysql://localhost/dualcore \`  
` --username root --password cloudera \`  
` --fields-terminated-by '\t' \`  
` --hive-table dualcore.suppliers \`  download to this database  
` --hive-import`                      special command (also creates metadata, not just data



---
`hadoop fs -cat /dualcore/employees/part-m-00004 | head` to see how cols are separated

### Step 2 - External Table

(within hue)  
Create an external tables based on **employees**  
`DROP TABLE IF EXISTS employees;`  
`CREATE EXTERNAL TABLE dualcore.employees(
emp_id	STRING
fname	STRING
lname	STRING
address	STRING
city	STRING
state	STRING
zipcode	STRING
job_title	STRING
email	STRING
active	STRING
salary	INT)`  
`row format delimited
    fields terminated by '\t'`  need to do this to specify a different col separator  
`location '/dualcore/employees'`;

---

### Step 3 - Create Internal table


`CREATE table default.ratings (
posted	TIMESTAMP,
cust_id	INT,
prod_id	INT,
rating	TINYINT,
message	STRING)
ROW FORMAT delimited
FIELDS TERMINATED BY '\t';`

`describe formatted ratings;` to see info and where table is located (managed vs external)


---

### Step 3b - Upload data to internal

**With hadoop**  
`hadoop fs -put` $`ADIR/data/ratings_2012.txt \
 /user/hive/warehouse/ratings`


**With load data inpath**  
Put the data in dualcore  
`hadoop fs -put` $`ADIR/data/ratings_2013.txt /dualcore`

Verify its there  
`hadoop fs -ls /dualcore/ratings_2013.txt`

(within hue)  
`LOAD DATA INPATH '/dualcore/ratings_2013.txt' INTO TABLE ratings;`



---

### Step 4 - Creating table with complex fields

` CREATE TABLE loyalty_program
 (cust_id INT,
     fname STRING,
     lname STRING,
     email STRING,
     level STRING,
     phone MAP<STRING, STRING>,
     order_ids ARRAY<INT>,
     order_value STRUCT< min:INT, max:INT, avg:INT, total:INT>)
 ROW FORMAT DELIMITED
     FIELDS TERMINATED BY '|'`  
     `COLLECTION ITEMS TERMINATED BY ','`  for array and struct  
     `MAP KEYS TERMINATED BY ':';` for map
     
     ---
     
This is a `local` file, not in `HDFS`  
`LOAD DATA LOCAL INPATH '/home/cloudera/training_materials/analyst/exercises/data_mgmt/loyalty_data.txt' 
INTO TABLE loyalty_program;`

---

**Map**  
`from loyalty_program select phone['HOME']
where cust_id = 1200866;`

**Array**  
`from loyalty_program select phone[2]
where cust_id = 1200866;`

**Struct**  
`from loyalty_program select ordervalue.total
where cust_id = 1200866;`


---

# Lab 8x - csv from the internet

[Lab](https://pages.github.umn.edu/deliu/bigdata19/04-Hive2/lab09-hiveopt.html)    
[Solution](https://pages.github.umn.edu/deliu/bigdata19/04-Hive2/lab09-hiveopt-solution.html)

**Download file**  
`wget http://idsdl.csom.umn.edu/c/share/MSBA6330/titanic.csv`


**Create the managed table**  
`CREATE TABLE titanic
(passengerid int,survived int,pclass int,name string,
sex string, age int,sibsp int,parch int,ticket string,
fare string,cabin string,embarked string)`

`ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (`  
`"separatorChar"=",",`  -- default: comma separated  
`"quoteChar"="\"",` -- default: use double quotes as quoteChar  
`"escapeChar"="\\"`  -- using \ as escapeChar.  
)  
`tblproperties ("skip.header.line.count"="1");` -- skip 1 header line


**Load the data in**  
`pwd` to see where the data was downloaded  
`load data local inpath "/home/cloudera/titanic.csv" into table titanic;`


---

# Lab 9 (Sept 25) - Hive Optimization

[Lab](https://pages.github.umn.edu/deliu/bigdata19/04-Hive2/lab09-hiveopt.html)  
[Solution](https://pages.github.umn.edu/deliu/bigdata19/04-Hive2/lab09-hiveopt-solution.html)


create directory  
`cd ADIR/exercises/
mkdir hive_optimization
cd hive_optimization`

download data  
`wget http://idsdl.csom.umn.edu/c/share/msba6330/movie.zip`

Create a Hadoop directory /user/cloudera/movies:  
`hadoop fs -mkdir /user/cloudera/movies`

unzip data and put into HDFS - avoids downloading to local first  
`unzip -p movie.zip | hadoop fs -put - /user/cloudera/movies/movie.txt`  
Note that `p` gets rids of messages  

how many lines?  
`hadoop fs -ls movies`

---

### Step 1 - Standalone

**Using hue**  
Run query - top 10 brands by sales  
`SELECT brand, COUNT(prod_id) AS num
FROM products
GROUP BY brand
ORDER BY num DESC
LIMIT 10;`

prefix the above with `explain` to see steps

Enable local mode  
`SET mapreduce.framework.name=local;`

Go back to mapreduce mode  
`SET mapreduce.framework.name=yarn;` 

---

### Step 3 - ORC

Create database & table  
 `create database movies;
 CREATE EXTERNAL TABLE movies.movie_raw (
     id INT,
     name STRING,
     year int
     ) ROW FORMAT DELIMITED  FIELDS TERMINATED BY '|' LOCATION '/user/cloudera/movies/';`


Now create a hive-managed ORC table movie_orc.  
 `CREATE TABLE movies.movie_orc (
     id INT,
     name STRING,
     year int
     ) STORED AS ORC;`


Load data into the orc table.  
` INSERT INTO TABLE movie_orc 
 SELECT *
 FROM movie_raw;`

---

#### Compare Disk size

`-du` = calc disk size  
`-h` = argument for du

**External table**  
`hadoop fs -du -h /user/cloudera/movies`  
 43.6 M  43.6 M  /user/cloudera/movies/movie.txt

**Managed table**  
` hadoop fs -du -h /user/hive/warehouse/movies.db`  
 15.9 M  15.9 M  /user/hive/warehouse/movies.db/movie_orc
 
Run a query  
`select year, count(*) from movie_orc group by year`

---

### Step 4 - Using partitioned tables

Create partitioned table (3 fields + 1 virtual field)  
 `CREATE TABLE movies.movie_orc_partitioned (
     id INT,
     name STRING,
     year INT
 ) partitioned by (decade int) 
 STORED AS ORC;`

We are going to use **dynamic** because we dont currently have a decade column  
Enable nonstrict dynamic partitioning  
 `SET hive.exec.dynamic.partition = True;`  
 `SET hive.exec.dynamic.partition.mode = nonstrict;`

Loading data into movie_orc_partitioned.  
` INSERT INTO TABLE movies.movie_orc_partitioned PARTITION(decade) 
 SELECT id,name,year, floor(year/10)*10
 FROM movies.movie_orc;`

Compare the performance of the following two queries.  
` select count(*) from movie_orc where floor(year/10)*10=1950;
 select count(*) from movie_orc_partitioned where decade=1950;`

---

### Creating buckets from movie_orc

`create table movie_bucketed
(id INT,
name STRING,
year INT
) CLUSTERED BY (id)
into 10 buckets;`

`import overwrite table movie_bucketed
select * from movie_orc;`

Run a query on one of the buckets  
2nd bucket out of the first set of 5  
`select count(*) from movie_bucketed
tablesample (bucket 2 out of 5 on id);`
